In [1]:
! pip3 install featexp
! pip3 install xgboost

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Local imports,
from featexp import (
    get_trend_stats, 
    get_univariate_plots, 
    univariate_plotter
)

In [8]:
notebook_path = os.path.abspath("test.ipynb")
data_folder = "/home-credit-default-risk/"
train_data_file = "application_train.csv"
test_data_file = "application_test.csv"
print(os.path.dirname(notebook_path) + data_folder + train_data_file)

c:\git\intelligent-systems-practice\article/home-credit-default-risk/application_train.csv


In [20]:
# Functions for data preprocessing.
def get_nonull_dummy_data(application_train_raw, dummy_drop=["ORGANIZATION_TYPE"]):
    # Idenifying float cols with less nulls and imputing with mean.
    nulls = pd.isnull(application_train_raw).sum()
    less_nulls = nulls[(nulls < 3075) & (nulls != 0)].index
    less_nulls_float = []
    for i in range(len(less_nulls)):
        if application_train_raw[less_nulls[i]].dtype != "O":
            less_nulls_float.append(less_nulls[i])

    application_train_raw[less_nulls_float] = application_train_raw[
                                                less_nulls_float].fillna(application_train_raw[less_nulls_float].mean())

    # Idenifying float cols with high nulls and creating null_flag 
    # column and imputing with min-10.
    more_nulls = nulls[(nulls >= 3075)].index
    more_nulls_float = []
    for i in range(len(more_nulls)):
        if application_train_raw[more_nulls[i]].dtype != "O":
            more_nulls_float.append(more_nulls[i])

    application_train_raw[more_nulls_float] = application_train_raw[
                                                more_nulls_float].fillna(application_train_raw[more_nulls_float].min() - 100)

    # Get dummies. Drop some columns for now.
    try:
        application_train_raw.drop(columns=dummy_drop, axis=1, inplace=True)  # Try using later.
    except Exception:
        pass

    all_cols = application_train_raw.columns
    cat_cols = []
    for col in all_cols:
        if application_train_raw[col].dtype == "O":
            cat_cols.append(col)

    application_train_raw = pd.get_dummies(application_train_raw, columns=cat_cols, dummy_na=True)

    return application_train_raw

In [21]:
def import_and_create_train_test_data(train_data_file, test_size=0.33, random_state=42):
    application_raw = pd.read_csv(train_data_file)
    application = get_nonull_dummy_data(application_raw, dummy_drop=["ORGANIZATION_TYPE"])

    X = application.drop(["TARGET"], axis=1)  # Contains ID.
    y = application["TARGET"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return (X_train, X_test, y_train, y_test)

In [22]:
def import_and_create_TEST_data(test_data_file):
    application_raw = pd.read_csv(test_data_file)
    application = get_nonull_dummy_data(application_raw, dummy_drop=["ORGANIZATION_TYPE"])
    return application

In [23]:
X_train, X_test, y_train, y_tests = import_and_create_train_test_data(os.path.dirname(notebook_path) +                                                                                                          data_folder + train_data_file)
X_TEST = import_and_create_TEST_data(os.path.dirname(notebook_path) + data_folder + test_data_file)

try:
    drop=['CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown', 'SK_ID_CURR']
    X_train = X_train.drop(drop, axis=1)
    X_test = X_test.drop(drop, axis=1)
except Exception:
    pass


MemoryError: Unable to allocate 140. MiB for an array with shape (39999, 3678) and data type uint8

In [ ]:
print(X_train)